# NYC Taxi fare problem - Assignment 1
***
> ##### __Author__: _Vanshaj Lokwani_
> ##### __SBU ID__: _112025869_

In [1]:
# Importing the tools required
import pandas as pd
import numpy as np
import sklearn as skl
from matplotlib import pyplot as plt
import seaborn as sns

The dataset contains 55M rows and trying to load all of them at once causes the system to hang and intermittently crash. For the initial cleaning and testing phase, using a subset of the data. Once we have an idea of the actual cleaning effort required, we can iteratively perform that on the whole dataset for further processing. 

__For the time being, using first 10M rows as our subset. There is no reason behind this and the number was selected by intuition. Choosing 10M rows gives us enough liberty with the data to identify patterns and the data is small enough to not overload the RAM during pre-processing__

In [2]:
# Getting the dataset
NUM_ROWS = 10**7 # 10M ROWS
train_df = pd.read_csv('../dataset/train.csv', nrows=NUM_ROWS)

Displaying the first 5 rows from the dataframe to get a rough idea of the DF looks like and if it's loaded correctly. 

In [3]:
display(train_df.head())

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [4]:
# Few steps to make our life easier in the long run.
# set the new index as the 'key' coumn. The dataset specifies that the key is unique overall
# the it does not make sense to maintain two different indexes. 
train_df.set_index('key', inplace=True, drop=True) 

#getting the columns in a list. Might be helpful later on. 
columns = list(train_df.columns)
display(columns)

['fare_amount',
 'pickup_datetime',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude',
 'passenger_count']

## Question 1: 
##### ** Take a look at the training data. There may be anomalies in the data that you may need to factor in before you start on the other tasks. Clean the data first to handle these issues. Explain what you did to clean the data (in bulleted form). (10 pt) **


##### Let's print a summary of the data using the .describe() api provided in pandas. This may help us detect any anomalies that might exist in the system.

But before we get into that, we need to supress scientific notation in pandas. The easiest way to do this is to call 'set_option'. We'll set the precision for the number to be accurate upto 3 decimal places. More information on how to do that can be found at the stackoverflow link: https://stackoverflow.com/questions/21137150/format-suppress-scientific-notation-from-python-pandas-aggregation-results

In [5]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
display(train_df.describe())

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,10000000.000,10000000.000,10000000.000,9999931.000,9999931.000,10000000.000
mean,11.339,-72.508,39.919,-72.509,39.919,1.685
std,9.800,12.994,9.323,12.875,9.237,1.323
min,-107.750,-3439.245,-3492.264,-3426.601,-3488.080,0.000
25%,6.000,-73.992,40.735,-73.991,40.734,1.000
50%,8.500,-73.982,40.753,-73.980,40.753,1.000
75%,12.500,-73.967,40.767,-73.964,40.768,2.000
max,1273.310,3457.626,3344.459,3457.622,3351.403,208.000


#### We can already see that there are clearly some issues with the data. Let's pick the columns one by one.

__ColumnName__: *pickup_longitude, pickup_latitude, dropoff_longitude & dropoff_latitude* Possible Issues:

* dropoff_longitude and dropoff_latitude count is lesser compared to other columns. This signifies the presence of missing values or NaN values in the two columns. We would have to take care of these depending how many values are missing.

In [6]:
missing_long_indexes = train_df[train_df.dropoff_longitude.isnull().values].dropoff_longitude.index
missing_lat_indexes = train_df[train_df.dropoff_latitude.isnull().values].dropoff_latitude.index
missing_values_list = set(missing_long_indexes).union(set(missing_lat_indexes))
print("Number of rows with missing indexes: {}".format(len(missing_values_list)))

Number of rows with missing indexes: 69


the number seems small enough that these rows can be dropped without worrying about loosing important data. 69 in 10M is not a very significant number.  

In [7]:
train_df.drop(missing_values_list, inplace=True)

In [8]:
# describe api should return equal counts for all columns with no na values
display(train_df.describe())
display("Any null values left in the two columns: %s" % train_df[['dropoff_longitude', 'dropoff_latitude']].isnull().values.any())

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,9999931.000,9999931.000,9999931.000,9999931.000,9999931.000,9999931.000
mean,11.338,-72.508,39.919,-72.509,39.919,1.685
std,9.800,12.994,9.323,12.875,9.237,1.323
min,-107.750,-3439.245,-3492.264,-3426.601,-3488.080,0.000
25%,6.000,-73.992,40.735,-73.991,40.734,1.000
50%,8.500,-73.982,40.753,-73.980,40.753,1.000
75%,12.500,-73.967,40.767,-73.964,40.768,2.000
max,1273.310,3457.626,3344.459,3457.622,3351.403,208.000


'Any null values left in the two columns: False'

* The range of latitude values is -90 to 90 and the range of longitude values is -180 to 180. From the data summary, it's clear that the data does not follow the range. Calculating the number of points which are outside the range: 

In [9]:
# Out of Range = or_
or_pickup_longitude = set(train_df[(train_df.pickup_longitude < -180.0) | (train_df.pickup_longitude > 180.0)].index)
or_pickup_latitude = set(train_df[(train_df.pickup_latitude < -90.0) | (train_df.pickup_latitude > 90.0)].index)
or_dropoff_longitude = set(train_df[(train_df.dropoff_longitude < -180.0) | (train_df.dropoff_longitude > 180.0)].index)
or_dropoff_latitude = set(train_df[(train_df.dropoff_latitude < -90.0) | (train_df.dropoff_latitude > 90.0)].index)
or_indexes = or_pickup_longitude.union(or_pickup_latitude).union(or_dropoff_latitude).union(or_dropoff_longitude)
display("Number of rows with outside range values: %s" % len(or_indexes))

'Number of rows with outside range values: 476'

In [10]:
train_df.loc[list(or_indexes)].describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,476.000,476.000,476.000,476.000,476.000,476.000
mean,10.737,-316.692,116.937,-258.439,63.743,1.901
std,8.741,1090.612,1017.041,1075.646,1004.023,2.719
min,2.500,-3439.245,-3492.264,-3426.601,-3488.080,1.000
25%,5.700,-736.367,40.721,-736.237,40.704,1.000
50%,8.100,-73.993,40.757,-73.990,40.756,1.000
75%,11.700,-73.960,288.889,-73.953,40.811,2.000
max,57.330,3457.626,3344.459,3457.622,3351.403,51.000


476 in ~10M is a very small percentage of datapoints. We can look into the datapoints individually later and try to figure out if there is any way we can fix these outliers but for now dropping the points.

In [11]:
train_df.drop(or_indexes, inplace=True)

In [12]:
display(train_df.describe())

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,9999455.000,9999455.000,9999455.000,9999455.000,9999455.000,9999455.000
mean,11.339,-72.496,39.916,-72.500,39.918,1.685
std,9.800,10.465,6.123,10.449,6.117,1.323
min,-107.750,-168.604,-74.824,-173.342,-74.193,0.000
25%,6.000,-73.992,40.735,-73.991,40.734,1.000
50%,8.500,-73.982,40.753,-73.980,40.753,1.000
75%,12.500,-73.967,40.767,-73.964,40.768,2.000
max,1273.310,154.101,89.742,154.101,81.555,208.000



__ColumnName__: *fare_amount*
Possible Issues: 
* fare_amount is negative which should not be possible. Assumption: there was an issue with data entry or fare calculation. looking into the dataset to fetch all the rows where fare_amount is negetive.

In [13]:
neg_fare = train_df[train_df.fare_amount < 0]
display(neg_fare.describe())

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,420.000,420.000,420.000,420.000,420.000,420.000
mean,-9.288,-69.033,38.031,-67.975,37.448,1.848
std,15.072,18.472,10.177,20.198,11.127,1.433
min,-107.750,-74.230,0.000,-74.230,0.000,0.000
25%,-6.200,-73.990,40.727,-73.989,40.725,1.000
50%,-3.500,-73.976,40.753,-73.974,40.750,1.000
75%,-2.500,-73.951,40.765,-73.943,40.764,2.000
max,-0.010,0.000,40.919,0.000,40.919,6.000


Looking at the number of data points that have negative fare value is just 420 in ~10M. We can safely drop these values from the dataset as well.

In [14]:
train_df.drop(neg_fare.index, inplace=True)
display(train_df.head())
display(train_df.describe())

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
key,,,,,,,
2009-06-15 17:26:21.0000001,4.500,2009-06-15 17:26:21 UTC,-73.844,40.721,-73.842,40.712,1
2010-01-05 16:52:16.0000002,16.900,2010-01-05 16:52:16 UTC,-74.016,40.711,-73.979,40.782,1
2011-08-18 00:35:00.00000049,5.700,2011-08-18 00:35:00 UTC,-73.983,40.761,-73.991,40.751,2
2012-04-21 04:30:42.0000001,7.700,2012-04-21 04:30:42 UTC,-73.987,40.733,-73.992,40.758,1
2010-03-09 07:51:00.000000135,5.300,2010-03-09 07:51:00 UTC,-73.968,40.768,-73.957,40.784,1


,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,9999035.000,9999035.000,9999035.000,9999035.000,9999035.000,9999035.000
mean,11.339,-72.496,39.916,-72.500,39.918,1.685
std,9.799,10.464,6.123,10.448,6.117,1.323
min,0.000,-168.604,-74.824,-173.342,-74.193,0.000
25%,6.000,-73.992,40.735,-73.991,40.734,1.000
50%,8.500,-73.982,40.753,-73.980,40.753,1.000
75%,12.500,-73.967,40.767,-73.964,40.768,2.000
max,1273.310,154.101,89.742,154.101,81.555,208.000


__ColumnName__: *passenger_count* Possible Issues:
* The maximum value of passenger_count reaches 208, which does not seem like it would be possible for a taxi. the maximum number of people allowed in a NYC taxi is 5 + a child below the age of 7. Let's take the case of a child in a taxi as a special case that we will deal with at a later point in time. Lets look at data where passenger count > 5.

In [16]:
train_df[train_df.passenger_count > 5.0].groupby('passenger_count').count()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
passenger_count,,,,,,
6,211829,211829,211829,211829,211829,211829
7,2,2,2,2,2,2
9,3,3,3,3,3,3
49,1,1,1,1,1,1
129,1,1,1,1,1,1
208,9,9,9,9,9,9


<p>The number of datapoints above 6 passengers is few in number. These can be considered as outliers and dropped safely. There is no point trying to fix the passenger counts because the change caused by these number will not be significant</p>

In [20]:
train_df.drop(train_df[train_df.passenger_count > 6].index, inplace=True)
display(train_df.head())

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
key,,,,,,,
2009-06-15 17:26:21.0000001,4.500,2009-06-15 17:26:21 UTC,-73.844,40.721,-73.842,40.712,1
2010-01-05 16:52:16.0000002,16.900,2010-01-05 16:52:16 UTC,-74.016,40.711,-73.979,40.782,1
2011-08-18 00:35:00.00000049,5.700,2011-08-18 00:35:00 UTC,-73.983,40.761,-73.991,40.751,2
2012-04-21 04:30:42.0000001,7.700,2012-04-21 04:30:42 UTC,-73.987,40.733,-73.992,40.758,1
2010-03-09 07:51:00.000000135,5.300,2010-03-09 07:51:00 UTC,-73.968,40.768,-73.957,40.784,1


In [21]:
display(train_df.describe())

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,9999019.000,9999019.000,9999019.000,9999019.000,9999019.000,9999019.000
mean,11.339,-72.496,39.916,-72.500,39.918,1.685
std,9.799,10.464,6.123,10.448,6.117,1.308
min,0.000,-168.604,-74.824,-173.342,-74.193,0.000
25%,6.000,-73.992,40.735,-73.991,40.734,1.000
50%,8.500,-73.982,40.753,-73.980,40.753,1.000
75%,12.500,-73.967,40.767,-73.964,40.768,2.000
max,1273.310,154.101,89.742,154.101,81.555,6.000
